In [182]:
import pandas as pd

csv_file = 'reports_df.csv'
df = pd.read_csv(csv_file)
df = df.drop_duplicates()

df.shape[0]

69674

In [ ]:
import re

finding = "Findings:\n\nLeft Ventricle: Normal size based on linear index dimension. Mildly decreased systolic function. Ejection fraction = 45 %. No regional wall motion abnormalities. Normal relative wall thickness and LV mass. Mild diastolic dysfunction. Normal filling pressures. No LV thrombus is seen.\nRight Ventricle: Normal size by linear dimension. Normal systolic function. RV wall thickness is normal.\nLeft Atrium: Moderately dilated by biplane volume index.\nRight Atrium: Normal size by volume index.\nMitral Valve: Normal valve leaflets. Mild annular calcification. Trivial regurgitation.\nTricuspid Valve: Normal valve leaflets. Normal mobility of the leaflets. Trivial regurgitation.\nAortic Valve: Trileaflet valve; sclerosis. Trivial valvular regurgitation.\nPulmonary Valve/Artery: Not well seen. Mild regurgitation. Normal pulmonary artery systolic pressure: 24 mmHg.\nAorta: Normal sinuses of Valsalva by index dimension. Normal sinuses of Valsalva by linear dimension. Normal proximal ascending aorta by index dimension. Normal proximal ascending aorta by linear dimension.\nVenous: Inferior vena cava is normal size with greater than 50% respiratory variation.\nPericardium/Other: Trivial pericardial effusion.\nShunts: Atrial septum bulges rightward. No shunt by color flow Doppler to suggest PFO/ASD.\n \n"
finding = df['findings_txt'][0]
finding = "Findings:\n\nLeft Ventricle: Normal size based on linear index dimension. Mildly decreased systolic function. Ejection fraction = 45 %. No regional wall motion abnormalities. Normal relative wall thickness and LV mass. Mild diastolic dysfunction. Normal filling pressures. No LV thrombus is seen.\nRight Ventricle: Normal size by linear dimension. Normal systolic function. RV wall thickness is normal.\nLeft Atrium: Moderately dilated by biplane volume index.\nRight Atrium: Normal size by volume index.\nMitral Valve: Normal valve leaflets. Mild annular calcification. Trivial regurgitation.\nTricuspid Valve: Normal valve leaflets. Normal mobility of the leaflets. Trivial regurgitation.\nAortic Valve: Trileaflet valve; sclerosis. Trivial valvular regurgitation.\nPulmonary Valve/Artery: Not well seen. Mild regurgitation. Normal pulmonary artery systolic pressure: 24 mmHg.\nAorta: Normal sinuses of Valsalva by index dimension. Normal sinuses of Valsalva by linear dimension. Normal proximal ascending aorta by index dimension. Normal proximal ascending aorta by linear dimension.\nVenous: Inferior vena cava is normal size with greater than 50% respiratory variation.\nPericardium/Other: Trivial pericardial effusion.\nShunts: Atrial septum bulges rightward. No shunt by color flow Doppler to suggest PFO/ASD.\n \n"
unique_sections = []
unique_findings = {}

# Remove the initial 'Findings:' part
finding = finding.replace('Findings:', '').strip()

# Split the string into sections
sections = re.split(r'\n(?=[A-Z])', finding)

for section in sections:
    if ':' in section:  # only process sections that contain findings
        # Extract section name and findings
        section_name, section_findings = section.split(':', 1)
        section_name = section_name.strip()

        # Add section name to unique_sections
        unique_sections.append(section_name)

        # Clean up section_findings and split them into individual findings by period
        section_findings = [f.strip() for f in section_findings.split('.') if f.strip() != '']

        # Remove any numeric data
        section_findings = [re.sub(r'\d+.*', '', f) for f in section_findings]

        # If the section does not exist yet in the dictionary, add it
        if section_name not in unique_findings:
            unique_findings[section_name] = set()

        # Add section_findings to unique_findings
        unique_findings[section_name].update(section_findings)

print("Unique sections:")
print(unique_sections)

print("\nUnique findings:")
for section_name, findings in unique_findings.items():
    print(section_name)
    for finding in findings:
        print(finding)
    print()


In [ ]:
import re
from collections import defaultdict

findings_text = df['findings_txt']
# findings_text = findings_text[:10]
section_frequencies = defaultdict(int)
unique_findings = defaultdict(lambda: defaultdict(int))

for finding_text in findings_text:
    # Remove the initial 'Findings:' part
    finding_text = finding_text.replace('Findings:', '').strip()

    # Replace double backslashes with actual newline characters
    finding_text = finding_text.replace('\\n', '\n')

    # Split the string into sections
    sections = re.split(r'\n(?=[A-Z])', finding_text)
    for section in sections:
        if ':' in section:  # only process sections that contain findings
            # Extract section name and findings
            section_name, section_findings = section.split(':', 1)
            section_name = section_name.strip()

            # Increment the count for this section name
            section_frequencies[section_name] += 1

            # Clean up section_findings and split them into individual findings by period
            section_findings = [f.strip() for f in section_findings.split('.') if f.strip() != '']

            # Remove any numeric data
            section_findings = [re.sub(r'\d+.*', '', f) for f in section_findings]

            # Increment the count for each finding within this section
            for finding in section_findings:
                unique_findings[section_name][finding] += 1


In [ ]:

print("Section frequencies:")
for section, count in section_frequencies.items():
    print(f"{section}: {count}")

print("\nUnique findings:")
for section_name, findings in unique_findings.items():
    print(section_name)
    for finding, count in findings.items():
        print(f"{finding}: {count}")
    print()


In [ ]:
print("Section frequencies (by descending count):")
for section, count in sorted(section_frequencies.items(), key=lambda item: item[1], reverse=True):
    print(f"{section}: {count}")

In [ ]:
print("\nUnique findings (by descending count): \n")
for section_name, findings in unique_findings.items():
    print(section_name + "\n")

    sorted_findings = sorted(findings.items(), key=lambda item: item[1], reverse=True)
    top_50_findings = sorted_findings[:50]
    total_count = sum(findings.values())
    top_50_count = sum(count for finding, count in top_50_findings)

    for finding, count in top_50_findings:
        print(f"{finding}: {count}")

    percentage = (top_50_count / total_count) * 100 if total_count > 0 else 0
    print(f"\nThe top 50 findings make up {percentage:.2f}% of the total.\n")


In [ ]:
# For sections
total_section_counts = sum(section_frequencies.values())
over_1000_sections = {section: count for section, count in section_frequencies.items() if count > 1000}
over_1000_section_counts = sum(over_1000_sections.values())
percentage_over_1000_section_counts = (over_1000_section_counts / total_section_counts) * 100
percentage_over_1000_sections_keys = (len(over_1000_sections) / len(section_frequencies)) * 100

print(f"Percentage of total section count taken by sections with count over 1000: {percentage_over_1000_section_counts}%")
print(f"Percentage of section keys with count over 1000: {percentage_over_1000_sections_keys}%")

# For findings
total_finding_counts = sum(count for findings in unique_findings.values() for count in findings.values())
over_1000_findings = {section: {finding: count for finding, count in findings.items() if count > 1000} for section, findings in unique_findings.items()}
over_1000_finding_counts = sum(count for findings in over_1000_findings.values() for count in findings.values())
percentage_over_1000_finding_counts = (over_1000_finding_counts / total_finding_counts) * 100
percentage_over_1000_findings_keys = (sum(len(findings) for findings in over_1000_findings.values()) / sum(len(findings) for findings in unique_findings.values())) * 100

print(f"Percentage of total finding count taken by findings with count over 1000: {percentage_over_1000_finding_counts}%")
print(f"Percentage of finding keys with count over 1000: {percentage_over_1000_findings_keys}%")

In [ ]:
print("Section frequencies (alphabetically):")
for section, count in sorted(section_frequencies.items(), key=lambda item: item[0]):
    print(f"{section}: {count}")


In [ ]:
print("\nUnique findings (by descending count):")
for section_name, findings in unique_findings.items():
    print(section_name)

    sorted_findings = sorted(findings.items(), key=lambda item: item[1], reverse=True)
    top_50_findings = sorted_findings[:100]
    total_count = sum(findings.values())
    top_50_count = sum(count for finding, count in top_50_findings)

    for finding, count in top_50_findings:
        print(f"{finding}: {count}")

    percentage = (top_50_count / total_count) * 100 if total_count > 0 else 0
    print(f"\nThe top 100 findings make up {percentage:.2f}% of the total.\n")


In [191]:
import re

measurement_counts = {}

for index, row in df.iterrows():
    text = row['calcs_txt'].replace('\\n', '  ')
    matches = re.findall(r'  (.*?)(:|\')', text)
    
    for match, boundary in matches:
        # Cut off any text up until a closing parenthesis ")" if it exists
        closing_parenthesis_index = match.rfind(')')
        if closing_parenthesis_index != -1:
            match = match[closing_parenthesis_index+1:]
        
        last_space_index = match.rfind('  ')
        if last_space_index != -1:
            match = match[last_space_index+2:]
        
        # Remove trailing or leading spaces and the colon
        measurement_name = match.strip().rstrip(':')
        
        if measurement_name in measurement_counts:
            measurement_counts[measurement_name] += 1
        else:
            measurement_counts[measurement_name] = 1


In [192]:
sorted_measurement_counts = {k.rstrip(':'): v for k, v in sorted(measurement_counts.items(), key=lambda item: item[1], reverse=True)}

# Print the top 100 items
top_100 = list(sorted_measurement_counts.items())[:100]
for measurement, count in top_100:
    print(measurement, count)


Left Atrium 136822
 105865
Left Ventricle 69568
LVIDd 69331
LVIDd index 69252
LVPWd 69250
Right Ventricle 69248
IVSd 69247
LV RWT 69245
Aorta 69232
LV Mass index 69068
LVIDs 69014
LV Diastolic Function 68739
RA Volume index 68709
Aorta Sinuses 68639
Aorta Sinuses index 68556
RVd A4C 67780
Tricuspid Valve and PA/RV Systolic Pressure 67262
MV Peak E 66895
LVOT diam 65903
Septal e 65744
Lateral e 65722
Average E/e 65021
Prox Asc Aorta index 64814
Decel Time 64469
TAPSE 63667
MV Peak A 57318
MV E/A ratio 57015
Prox Ascending Aorta 53628
RV S 37923
LA Volume index 29018
Aortic Valve 25233
LV EDV index 23533
LV ESV index 23412
AoV Vmax 18002
AoV Peak PG 17936
SV LVOT 17185
Mitral Valve 14245
Prox Asc Aorta 11222
CO LVOT 11163
Cardiac Index 11126
TV S 10774
AR P1/2T 9885
Pulmonary Regurgitation 8641
PR EDV 8641
MV Mean Grad 8585
AoV Mean PG 7507
Aov Mean PG 7483
MR EROA 6498
Right Atrium 5017
Right Atrium index 5010
MR Volume 4938
PLAX RVOT Diam 2904
AR EROA 2070
Prox RVOT Diam 2051
AR Volume

In [180]:
text = df['calcs_txt'][0].replace('\\n', '  ')

matches = re.findall(r'  (.*?)(:|\')', text)

modified_matches = []

for match, boundary in matches:
    last_space_index = match.rfind('  ')
    if last_space_index != -1:
        modified_match = match[last_space_index+2:]
    else:
        modified_match = match
    modified_matches.append(modified_match + boundary)

print(modified_matches)


['Left Ventricle:', 'Left Atrium:', 'IVSd:', 'Left Atrium:', 'LVPWd:', 'LA vol index (Biplane):', 'LVIDd:', 'Right Ventricle:', 'LVIDd index:', 'RVd A4C:', 'LVIDs:', "RV S'", 'LV EF (Biplane):', 'TAPSE:', 'LV Mass index:', 'LV RWT:', 'RA Volume index:', 'LV EDV index:', 'Aorta:', 'LV ESV index:', 'Aorta Sinuses:', 'LVOT diam:', 'Aorta Sinuses index:', 'LV Diastolic Function:', 'Prox Ascending Aorta:', 'Prox Asc Aorta index:', 'MV Peak E:', 'Decel Time:', "Lateral e'", "Septal e'", "Average E/e'", 'Tricuspid Valve and PA/RV Systolic Pressure:']


In [179]:
print(df['calcs_txt'][0].replace('\\n','\n'))

Measurements and Calculations:
Left Ventricle:             Normal   Left Atrium:                      Normal
IVSd:            10 mm      (6-10)   Left Atrium:            39 mm     (30-40)
LVPWd:           10 mm      (6-10)   LA vol index (Biplane): *44 ml/m\xb2 (16-34)
LVIDd:           51 mm      (42-58)  Right Ventricle:            Normal
LVIDd index:     24 mm/m\xb2   (22-30)  RVd A4C:         34 mm      (25-41)
LVIDs:           39 mm      (25-40)  RV S'            10.5 cm/s  (=10)
LV EF (Biplane): 45 %       (52-72)  TAPSE:           20 mm      (>17)
LV Mass index:   88.9 g/m\xb2  (49-115) Right Atrium              Normal
LV RWT:          0.38       (<0.43)  RA Volume index: 23 ml/m\xb2 (18-39)
LV EDV index:    57.6 ml/m\xb2 (34-74)  Aorta:                         Normal
LV ESV index:    31.4 ml/m\xb2 (11-31)  Aorta Sinuses:        40 mm    (=40)
LVOT diam:       21 mm               Aorta Sinuses index:  19 mm/m\xb2 (=21)
LV Diastolic Function:               Prox Ascending Aorta: 32

In [ ]:
import pandas as pd

manual_string = raw_text
dataframe_value = df['calcs_txt'][0]

# Check the lengths of the strings
if len(manual_string) != len(dataframe_value):
    print("Lengths of the strings are different.")

# Compare the hexadecimal representations of the characters
for i, (ch1, ch2) in enumerate(zip(manual_string, dataframe_value)):
    if ch1.encode().hex() != ch2.encode().hex():
        print(f"Difference found at index {i}: '{ch1}' (manual string) vs '{ch2}' (DataFrame value)")


In [ ]:
import matplotlib.pyplot as plt

# Use the boolean mask to filter the DataFrame
date_df = sample_df[mask]

# Extract the years from the 'full_text' column
years = date_df['full_text'].str.extract(date_pattern)[0].str[-4:]

# Convert the extracted years to integers
years = years.astype(int)

# Plot a histogram of the years
plt.hist(years, bins=range(years.min(), years.max()+1), edgecolor='black', alpha=0.7)
plt.title('Distribution of Years')
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.show()


In [ ]:
print(years[years >= 2020].count())

In [ ]:
# Display the filtered DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

first_row = df.iloc[0].to_dict()

for col, value in first_row.items():
    if isinstance(value, str):  # check if the value is a string
        value = value.encode('utf-8').decode('unicode_escape')  # interpret the special characters
    print(f"{col}:\n{value}\n")

In [ ]:
import pandas as pd

def parse_measurements(text):
    lines = text.split("\n")
    data = {}
    for line in lines:
        parts = [part.strip() for part in line.split(':', 1)]
        if len(parts) == 2:
            key, value = parts
            value_parts = value.rsplit('(', 1)
            if len(value_parts) == 2:
                value_parts[1] = value_parts[1].rstrip(')')
                data[key] = value_parts[0].strip()
                data[key + ' normal range'] = value_parts[1]
            else:
                data[key] = value_parts[0]
    return data

parsed_data = df['calcs_txt'].apply(parse_measurements)
parsed_df = pd.DataFrame(list(parsed_data))
